# Imports

In [7]:
#dataframe
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json

# Data Gathering

### Parse the sitemap to gather URLs

In [6]:
content = []
#Read the xml file
def extract_recipe_locs(file):
    with open(file,'r',encoding = 'utf-8') as file:    
        #Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        #Combine the lines in the list into a string
        content = ''.join(content)
        soup_xml = BeautifulSoup(content, 'lxml')
        result = soup_xml.find_all('loc')
        result = [a.text for a in list(result) if "recipe" in a.text]
        return result

### Scrape from target URL

In [7]:
def scrape(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

### Extract Relevant Data

In [8]:
def extract_first_tag(s):
    pattern = r"({\"HasSponsor\".*};)"
    for script in s.find_all('script'):
        if script.string:    
            dataKeys_match = re.search(pattern, script.string, re.DOTALL)
        if dataKeys_match:
            dataKeys_text = dataKeys_match.group(0)
            dataKeys_text = "{\"information\":" + dataKeys_text[:-1] +"}"    
            print("dataKeys:", dataKeys_match.group(0))
            return json.loads(dataKeys_text)  
        
#extracts data from two relevant json vars in the html        
def extract_data(s):
    try:
        second_tag = s.find('script', {'type': 'application/ld+json'})    
        data_dict = json.loads(second_tag.string)

        third_tag = s.find('script', {'type': 'application/json'})
        if third_tag:
            data_dict_third = json.loads(third_tag.string)

            ret_dict = {
                "title" :data_dict['name'],
                "description": data_dict['description'],
                "category": data_dict['recipeCategory'],
                "total_time" : data_dict['totalTime'],
                #IN LIST FORMAT
                "ingredients" : data_dict["recipeIngredient"],
                #IN LIST FORMAT
                "instruction" :[a["text"] for a in data_dict["recipeInstructions"]],
                "image_src": data_dict["image"],
                "total_time2": data_dict_third['pageContext']['pageProps']['config']['totalTime'],
                "difficulty": data_dict_third['pageContext']['pageProps']['config']['tagDifficulty']['title']
            }
            return ret_dict
        else:
            print("THIRD JSON SECTION MISSING")
            return None 
    except Exception as e:
        print("some error happened: ", e)

## Scrape all the data

In [9]:
gbc_urls = extract_recipe_locs(r'C:\Users\arron\Desktop\recipe_classifier_proj\site_files\sitemaps\gbc_sitemap.xml')
gic_urls = extract_recipe_locs(r'C:\Users\arron\Desktop\recipe_classifier_proj\site_files\sitemaps\italian_sitemap.xml')
urls = gbc_urls + gic_urls

data = []
for each in urls: 
    raw = scrape(each)
    formatted = extract_data(raw)
    if formatted != None:
        data.append(formatted)

some error happened:  'NoneType' object has no attribute 'string'


KeyboardInterrupt: 

# Data loading and formatting

### Load data from csv

In [10]:
df = pd.read_csv("formatted_data_REMOVED_CHARS.csv")
try:
    df = df.drop('Unnamed: 0', axis=1)
    df = df.drop('total_time', axis=1)
except:
    print("attempted to delete non existent collumn")
    
cols = df.columns
df.head()

,title,description,category,ingredients,instruction,image_src,total_time2,difficulty
0,"Veal sweetbreads with ras el hanout, cauliflow...",Marcus Eaves' beautiful sweetbread dish master...,Starter,"100g of veal sweetbreads, 70g of T45 flour, 5g...",Start by making the cauliflower crisp to garni...,https://media-cdn2.greatbritishchefs.com/media...,90,medium
1,"Cured sea trout with garden pea, nasturtium an...",Phil Fanning celebrates spring's finest fish w...,Main,"250g of 00 flour, 150g of egg yolk, 15g of mil...","To make the pasta, bring all of the ingredient...",https://media-cdn2.greatbritishchefs.com/media...,150,challenging
2,"Grilled fillets of sea bass with herb risotto,...",This grilled sea bass recipe includes a bed of...,Main,"4 sea bass fillets, each weighing 140g, ground...","For the herb risotto, melt the butter in a wid...",https://media-cdn2.greatbritishchefs.com/media...,45,medium
3,Orange and pink risotto,Josh Eggleton delivers a gorgeous risotto reci...,Main,"150g of pearl barley, 4 carrots, 4 red beetroo...","In a juicer, juice 2 of the carrots and set as...",https://media-cdn2.greatbritishchefs.com/media...,90,medium
4,Calçots with romesco sauce,A quintessential Catalan pairing that's stood ...,Starter,"12 calçots, olive oil, for drizzling, sea sal...",Preheat an oven to 170 C/150 C fan/gas mark 3....,https://media-cdn2.greatbritishchefs.com/media...,30,easy
